# Generate VWAP using dedicated service - C#

### Overview
The Volume Weighted Average price is similar to a moving average, except volume is included to weight the average price over a one trading day period.

It serves as a reference point for one trading day prices. Comparing current prices with the VWAP values enables to:
* mesure trading efficiency by comparing individual trades to VWAP
* highlight liquidity points as vwap reflects prices levels weighted by volume
* determine the intraday trend
* etc.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** `intraday` VWAP price.

### Inputs/outputs
VWAP indicator requires instrument's identifier, date time intervals and time range as per inputs and returns the VWAP data points.

VWAP is a cumulative indicator which means the number of data points progressively increases throughtout the day.

This sample shows how to plot a simple intraday VWAP graph for basis technical analysis using an open source library. 

### Services used
This sample uses gRPC requests in order to retrieve VWAP indicator from the hosted service. The queried endpoint in this script are:
* *TickVwapService*: to directly retrieve intraday VWAP objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run VWAP sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Core;
using Grpc.Net.Client;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *VWAP* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Vwap request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [ ]:
var today = DateTime.Today;
var dateIntervals = new DateInterval
{
    StartDate = new Date { Year = 2022, Month = 02, Day = 18 },
    EndDate = new Date { Year = 2022, Month = 02, Day = 18 }
};

var timeInterval = new TimeInterval
{
    StartTime = new TimeOfDay { Hours = 08, Minutes = 00, Seconds = 00 },
    EndTime = new TimeOfDay { Hours = 20, Minutes = 00, Seconds = 00 }
};

#### 3.3 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints();
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval); 

The following code snippets create requests for Bollinger bands:

In [ ]:
// Generate tick VWAP request
var request = new TickVwapRequest() 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Constraints = constraints, 
};

// Instantiate tick VWAP service
var service = new TickVwapService.TickVwapServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve VWAP

In [ ]:
var vwaps = new List<TickVwapResponse>();

var call = service.TickVwap(request, headers);
await foreach (var data in call.ResponseStream.ReadAllAsync())
{
    vwaps.Add(data);
}

//display(vwaps)

#### 4.3 Plot VWAP and trades
Display *VWAP* and *trade prices* over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graphs for *VWAP* and *trade prices* as follows:

In [ ]:
var vwapGraph = new List<Scattergl>
{
    new Scattergl
    {
        name = "Trade",
        mode = "lines",
        x = vwaps.Select(elt => elt.TimeStamp.ToDateTime()),
        y = vwaps.Select(elt => elt.Trade)
    },
    new Scattergl
    {
        name = "Vwap",
        mode = "lines",
        x = vwaps.Select(elt => elt.TimeStamp.ToDateTime()),
        y = vwaps.Select(elt => elt.Vwap)
    }
};

var chart = Chart.Plot(vwapGraph);
chart.WithTitle($"Vwap and Trades for {ticker}");
chart.WithYTitle("Price");
chart.WithXTitle("Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);